In [27]:
import pickle
import random
import string
import time
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [ ]:
import os
os.chdir('../movies')
from movieLens import MovieLens

ml = MovieLens()

# Nuevos usuarios

Vamos a asociar un nombre de usuario y contraseña a cada uno de los userIds existentes en el dataset de ratings

In [28]:
def create_user_password_dict(num_users):
    users = {}
    for i in range(num_users):
        user = f"user{i + 1}"
        password = ''.join(random.choices(string.ascii_letters + string.digits, k=12))
        users[i + 1] = {"user": user, "password": password}
    return users

num_users = 610
user_password_dict = create_user_password_dict(num_users)

# Show users' information
for user_id, data in user_password_dict.items():
    print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")


ID: 1 | User: user1 | Password: zzn8FF53vdWx
ID: 2 | User: user2 | Password: wCLI1MQBHJv4
ID: 3 | User: user3 | Password: ebou3Ubzyxdo
ID: 4 | User: user4 | Password: ogYiHdlKBvgY
ID: 5 | User: user5 | Password: eZANXM8HmmSe
ID: 6 | User: user6 | Password: lizSToux0SS3
ID: 7 | User: user7 | Password: t8pzMadE9a24
ID: 8 | User: user8 | Password: QFaPKhbu5dJL
ID: 9 | User: user9 | Password: RwYh0fh8f3yd
ID: 10 | User: user10 | Password: 8be0PfR6b2tP
ID: 11 | User: user11 | Password: IPQPknMBCgR6
ID: 12 | User: user12 | Password: cDRRbPX6z3yS
ID: 13 | User: user13 | Password: mzi3XG9SuV7X
ID: 14 | User: user14 | Password: KXLAQoFPogIW
ID: 15 | User: user15 | Password: 2hbyKGJkElaZ
ID: 16 | User: user16 | Password: sl1tBGyjOLqM
ID: 17 | User: user17 | Password: R0a8bZNaVchC
ID: 18 | User: user18 | Password: QCAKQMtMu51o
ID: 19 | User: user19 | Password: E1Y5syhpoHtz
ID: 20 | User: user20 | Password: 5AxQDFbyj6rR
ID: 21 | User: user21 | Password: m7zVmKnioXzz
ID: 22 | User: user22 | Passwor

In [29]:
# Change the password of users 1 and 4
user_password_dict[1]["password"] = "123"
user_password_dict[4]["password"] = "456"

# # Show users' information
# for user_id, data in user_password.items():
#     print(f"ID: {user_id} | User: {data['user']} | Password: {data['password']}")

In [30]:
# Save the dictionary
with open("../dashboard/user_password_dict.pkl", 'wb') as file:
        pickle.dump(user_password_dict, file)

Añadir nuevos usuarios al diccionario

In [ ]:
def add_new_user(username, password, users):
    for data in users.values():
        if data["user"] == username:
            print(f"El nombre de usuario '{username}' ya está en uso.")
            return

    # Obtener el último ID existente y aumentarlo en 1
    last_id = max(users.keys())
    new_id = last_id + 1

    # Agregar el nuevo usuario al diccionario
    users[new_id] = {"user": username, "password": password}
    print(f"Nuevo usuario agregado: ID: {new_id} | User: {username} | Password: {password}")

# Ejemplo de uso
username = input("Introduce el nombre de usuario: ")
password = input("Introduce la contraseña: ")

add_new_user(username, password, user_password_dict)

In [ ]:
user_password_dict

In [ ]:
existing_usernames = [user['user'] for user in user_password_dict.values()]
existing_usernames

In [ ]:
def existing_username(username,user_password_dict):
    return username in [user['user'] for user in user_password_dict.values()]

existing_username("user5555",user_password_dict)

In [31]:
# Load user-password database
with open('../dashboard/user_password_dict.pkl', 'rb') as file:
    user_password_dict = pickle.load(file)

In [32]:
user_password_dict

{1: {'user': 'user1', 'password': '123'},
 2: {'user': 'user2', 'password': 'wCLI1MQBHJv4'},
 3: {'user': 'user3', 'password': 'ebou3Ubzyxdo'},
 4: {'user': 'user4', 'password': '456'},
 5: {'user': 'user5', 'password': 'eZANXM8HmmSe'},
 6: {'user': 'user6', 'password': 'lizSToux0SS3'},
 7: {'user': 'user7', 'password': 't8pzMadE9a24'},
 8: {'user': 'user8', 'password': 'QFaPKhbu5dJL'},
 9: {'user': 'user9', 'password': 'RwYh0fh8f3yd'},
 10: {'user': 'user10', 'password': '8be0PfR6b2tP'},
 11: {'user': 'user11', 'password': 'IPQPknMBCgR6'},
 12: {'user': 'user12', 'password': 'cDRRbPX6z3yS'},
 13: {'user': 'user13', 'password': 'mzi3XG9SuV7X'},
 14: {'user': 'user14', 'password': 'KXLAQoFPogIW'},
 15: {'user': 'user15', 'password': '2hbyKGJkElaZ'},
 16: {'user': 'user16', 'password': 'sl1tBGyjOLqM'},
 17: {'user': 'user17', 'password': 'R0a8bZNaVchC'},
 18: {'user': 'user18', 'password': 'QCAKQMtMu51o'},
 19: {'user': 'user19', 'password': 'E1Y5syhpoHtz'},
 20: {'user': 'user20', 'pass

In [ ]:
def get_user_id(username, user_password_dict):
    for user_id, user_data in user_password_dict.items():
        if user_data['user'] == username:
            return user_id
    return None  # Return None if the username is not found

user_id = get_user_id('user1', user_password_dict)
user_id

# QUIZ. Obtener las películas con mayor número de valoraciones

Las 15 películas más valoradas se mostrarán a un nuevo usuario tras registrarse con el fin de conocer de forma superficial sus gustos cinematográficos y poder realizar recomendaciones personalizadas

In [ ]:
ratings = ml.ratings.copy()
ratings.head()

In [ ]:
k = 15
top_movies = ratings.groupby('movieId').size().nlargest(k).index.tolist()
top_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in top_movies]
top_movies_titles

Vamos a adaptar el nombre de las películas para que sea más sencillo de leer

In [ ]:
top_movies_titles = ['Forrest Gump',
 'The Shawshank Redemption',
 'Pulp Fiction',
 'The Silence of the Lambs',
 'The Matrix',
 'Star Wars: Episode IV - A New Hope',
 'Jurassic Park',
 'Braveheart',
 'Terminator 2: Judgment Day',
 "Schindler's List",
 'Fight Club',
 'Toy Story',
 'Star Wars: Episode V',
 'The Usual Suspects',
 'American Beauty']

In [ ]:
top_movies_dict = {}
for movie_id, movie_title in zip(top_movies, top_movies_titles):
    top_movies_dict[movie_id] = movie_title
    
top_movies_dict

In [ ]:
# Save the dictionary
with open("../dashboard/top_movies_dict.pkl", 'wb') as file:
        pickle.dump(top_movies_dict, file)

# Recomendaciones

In [ ]:
name_to_movieID = ml.name_to_movieID.copy()

with open("../dashboard/name_to_movieID.pkl", 'wb') as file:
        pickle.dump(name_to_movieID, file)

Vamos a ver cuales son las películas mejor valoradas. Para ello primero seleccionamos las 100 películas con mayor número de valoraciones, y entre ellas se seleccionan aquellas con mejor valoración media

In [ ]:
most_rated_movies = ratings.groupby('movieId').count().sort_values(by='rating', ascending=False).nlargest(100, 'rating')
most_rated_movies = most_rated_movies.reset_index()
ids_most_rated_movies = most_rated_movies['movieId']
ids_most_rated_movies

# Filter the ratings dataframe to obtain the information about the most rated mvoies
filtered_df = ratings[ratings['movieId'].isin(ids_most_rated_movies)]

# Compute the average rating of the movies
sorted_ratings = filtered_df.groupby('movieId').mean('rating').sort_values(by='rating', ascending=False).nlargest(10, 'rating')
sorted_ratings = sorted_ratings.reset_index()
sorted_movies = sorted_ratings['movieId']


sorted_movies_titles = [ml.movieID_to_name.get(movie_id, 'Unknown') for movie_id in sorted_movies]
sorted_movies_titles

sorted_movies_dict = {}
for movie_id, movie_title in zip(sorted_movies, sorted_movies_titles):
    sorted_movies_dict[movie_id] = movie_title
    
sorted_movies_dict

In [ ]:
# Common elements in top_movies and sorted_movies
list(set(top_movies_titles) & set(sorted_movies_titles))

Las dos películas más populares ('The Shawshank Redemption' and 'The Godfather) se recomendarán a todos los usuarios nuevos, dado que son populares entre los usuarios actuales del sistema y no se dispone información acerca de si han visto o no la película. Por otro lado, para el caso de las películas 'Fight Club', 'Schindler's List' y'Star Wars: Episode IV - A New Hope' se procede de diferente manera. Como estas películas se incluyen en el cuestionario inicial que debe rellenar el usuario solo se recomendarán en el caso de que el usuario las marque como no vistas. El resto de recomendaciones hasta llegar a 10 se obtienen calculando la distancia entre usuarios y seleccionando aquellas recomendaciones que se hagan al usuario con gustos más semejantes al nuevo usuario.

**¿Esta opción o mejor recomiendo las películas que más le hayan gustado al usuario parecido?**

## Cálculo de la distancia entre usuarios

En primer lugar vamos a añadir la información del nuevo usuario al conjunto de datos existente

In [3]:
ratings = pd.read_csv('../dashboard/ratings.csv')
ratings.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


Añadir el nuevo usuario a la base de datos

In [4]:
# Load new users' info
with open('../dashboard/valores_slider.pkl', 'rb') as file:
    valores_slider = pickle.load(file)
    
valores_slider

{356: 0,
 318: 0,
 296: 0,
 593: 0,
 2571: 0,
 260: 0,
 480: 0,
 110: 0,
 589: 0,
 527: 0,
 2959: 5,
 1: 1,
 1196: 0,
 50: 0,
 2858: 0}

In [5]:
userId = ratings['userId'].max()+1
timestamp = int(time.time())

# Add the slider information to the ratings dataframe
new_ratings = pd.DataFrame.from_dict(valores_slider, orient='index', columns=['rating'])
new_ratings.index.name = 'movieId'
new_ratings = new_ratings.reset_index()
new_ratings['userId'] = userId
new_ratings['timestamp'] = timestamp

# Reorder the columns in `df_new_ratings` to match the order of the existing DataFrame
new_ratings = new_ratings.reindex(columns=ratings.columns)
new_ratings

,userId,movieId,rating,timestamp
0,611,356,0,1688850648
1,611,318,0,1688850648
2,611,296,0,1688850648
3,611,593,0,1688850648
4,611,2571,0,1688850648
5,611,260,0,1688850648
6,611,480,0,1688850648
7,611,110,0,1688850648
8,611,589,0,1688850648
9,611,527,0,1688850648


In [6]:
# Concatenate the `ratings` DataFrame and `df_new_ratings` to add the new lines at the end
ratings = pd.concat([ratings, new_ratings], ignore_index=True)
ratings.to_csv('ratings.csv', index=False)
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100846,611,2959,5.0,1688850648
100847,611,1,1.0,1688850648
100848,611,1196,0.0,1688850648
100849,611,50,0.0,1688850648


In [ ]:
def add_new_user():
    # Load new users' info
    with open('../dashboard/valores_slider.pkl', 'rb') as file:
        valores_slider = pickle.load(file)
        
    #ratings = pd.read_csv('ratings.csv')
    
    userId = ratings['userId'].max()+1
    timestamp = int(time.time())
    
    

    # Add the slider information to the ratings dataframe
    new_ratings = pd.DataFrame.from_dict(valores_slider, orient='index', columns=['rating'])
    new_ratings.index.name = 'movieId'
    new_ratings = new_ratings.reset_index()
    new_ratings['userId'] = userId
    new_ratings['timestamp'] = timestamp

    # Reorder the columns in new_ratings to match the order of the ratings DataFrame
    new_ratings = new_ratings.reindex(columns=ratings.columns)
    
    # Concatenate the ratings DataFrame and new_ratings to add the new lines at the end
    ratings = pd.concat([ratings, new_ratings], ignore_index=True)
    
    return ratings
    
    # Save the updated file
    # ratings.to_csv('ratings.csv', index=False)

In [19]:
def find_most_similar_user(userId, distance_matrix):

    user_index = userId-1 # Restamos 1 porque los índices en Python comienzan desde 0

    # Calcular la distancia entre el usuario 611 y los demás usuarios
    user_distances = distance_matrix[user_index]

    # Encontrar el usuario más cercano (excluyendo al usuario mismo)
    most_similar_user_index = np.argmin(user_distances[np.nonzero(user_distances)])
    most_similar_user = most_similar_user_index + 1  # Sumamos 1 para obtener el índice real

    # Imprimir el índice del usuario más parecido
    print("El usuario más parecido al usuario "+str(userId)+" es:", most_similar_user) 
    
    with open('userId-to-recommend.pkl', 'wb') as archivo:
        pickle.dump(most_similar_user, archivo)

In [20]:
def compute_distance_matrix(ratings):
    # Create a pivot table
    pivot_table = pd.pivot_table(ratings, values='rating', index='userId', columns='movieId')

    # Replace NaN values with 0
    pivot_table = pivot_table.fillna(0)
    
    # Compute the distance between users
    distances = pdist(pivot_table.values, metric='euclidean')

    # Create the distance matrix
    distance_matrix = squareform(distances)
    most_similar_user_index = find_most_similar_user(ratings['userId'].max(),distance_matrix)

In [21]:
compute_distance_matrix(ratings)

El usuario más parecido al usuario 611 es: 442


# Recomendaciones usuarios existentes

In [ ]:
with open('../dashboard/recommendations.pkl', 'rb') as file:
        recommendations = pickle.load(file)
recommendations

In [ ]:
def get_movie_images(userId):

    numbers = [item[0] for item in recommendations[userId]]
    images = []
    for number in numbers:
        image_path = f"{number}.jpg"
        image = html.Img(src=dash.get_asset_url(image_path), style={"width": "110px", "height": "140px", "margin": "10px"})
        images.append(image)
    return images


In [ ]:
def get_user_preferences(userId, k, like=True):

    # Filter ratings for the specified userId and sort movies by rating in descending order
    user_ratings = ratings[ratings['userId'] == userId].copy()
    sorted_movies = user_ratings.sort_values(by='rating', ascending=False)

    if like:
        # Get the top k movies with highest ratings
        movieIds = list(sorted_movies.head(k).copy()['movieId'])
    
    # Get the top k movies with lowest ratings
    else:
        movieIds = list(sorted_movies.tail(k).copy()['movieId'])

    return movieIds

In [ ]:
numbers = get_user_preferences(1, 10, False)
numbers